## Chapter 7: Dependency Injection (DI) and The Repository Pattern

So far, we've built controllers that directly work with in-memory lists or simulated data. In a real application, you'll need to access databases, call external services, or implement complex business logic. How you structure these dependencies matters immensely for maintainability, testability, and flexibility. This chapter introduces two fundamental concepts: **Dependency Injection (DI)** – a design pattern that ASP.NET Core embraces natively, and the **Repository Pattern** – a common abstraction for data access. You'll learn how to use the built‑in DI container to manage your application's services, how to implement a clean repository layer, and how to inject these services into your controllers. By the end, you'll be writing loosely coupled, easily testable code that follows industry best practices.

### 7.1 Why Dependency Injection? (Loose Coupling, Testability)

To understand DI, let's first look at its opposite: **tight coupling**.

#### The Problem with Tight Coupling

Imagine a `ProductsController` that directly creates an instance of a `ProductRepository` class:

```csharp
public class ProductsController : Controller
{
    private readonly ProductRepository _repository;

    public ProductsController()
    {
        _repository = new ProductRepository(); // Controller creates its own dependency
    }

    public IActionResult Index()
    {
        var products = _repository.GetAll();
        return View(products);
    }
}
```

This seems simple, but it introduces several problems:

- **Hard to test**: To unit test `ProductsController`, you're forced to also use the real `ProductRepository`, which might hit a database. You cannot easily replace it with a mock or fake.
- **Inflexible**: If you later want to change the data source (e.g., from SQL to a cloud API), you have to modify the controller. The controller is tightly bound to a concrete implementation.
- **Violates the Single Responsibility Principle**: The controller now has the additional responsibility of instantiating its dependencies.

#### The Solution: Dependency Inversion Principle

The Dependency Inversion Principle (the "D" in SOLID) states:

> High-level modules should not depend on low-level modules. Both should depend on abstractions.

In practice, this means our controller should depend on an **interface** (abstraction), not a concrete class. The actual implementation is then **injected** from the outside—hence the name Dependency Injection.

```csharp
public class ProductsController : Controller
{
    private readonly IProductRepository _repository;

    public ProductsController(IProductRepository repository) // Dependency injected
    {
        _repository = repository;
    }
}
```

Now the controller doesn't know or care about the concrete repository. It only knows the contract (`IProductRepository`). This brings multiple benefits:

- **Testability**: You can pass a fake or mock repository during testing.
- **Flexibility**: You can swap implementations without changing the controller.
- **Separation of concerns**: The controller focuses on handling HTTP requests; the repository handles data access.

#### How DI is Implemented in ASP.NET Core

ASP.NET Core has a built‑in **Inversion of Control (IoC) container** that manages the creation and lifetime of objects. You register your services (classes and interfaces) with the container, and then the container automatically provides the correct instances when needed (a process called **service resolution**).

The container follows the **Hollywood Principle**: "Don't call us, we'll call you." Your code declares what it needs (via constructor parameters), and the container supplies it.

---

### 7.2 The Built-in DI Container

The DI container is configured in `Program.cs` (or `Startup.cs` in older templates). You register services with the `IServiceCollection` using extension methods.

#### Service Lifetimes

When you register a service, you must specify its **lifetime** – how long an instance lives and when it is reused. ASP.NET Core provides three lifetimes:

| Lifetime     | Description                                                                                                 | Typical Use Case                                  |
|--------------|-------------------------------------------------------------------------------------------------------------|---------------------------------------------------|
| **Transient** | A new instance is created **every time** it is requested.                                                  | Lightweight, stateless services.                  |
| **Scoped**   | A new instance is created **per client request** (connection). Within a single HTTP request, the same instance is reused. | Database contexts (e.g., Entity Framework DbContext), per-request services. |
| **Singleton** | A single instance is created **for the entire application lifetime** and shared across all requests.        | Caching services, logging, configuration objects. |

**Important:** The choice of lifetime affects performance and correctness. For example, a scoped service cannot depend on a transient service that lives longer (the container will throw an exception if you try). A general rule: use scoped for most of your business and data services, transient for stateless utilities, and singleton for truly shared resources.

#### Registering Services

You register services in `Program.cs` before `var app = builder.Build();`.

```csharp
var builder = WebApplication.CreateBuilder(args);

// Add services to the container.
builder.Services.AddControllersWithViews();

// Register your own services
builder.Services.AddScoped<IProductRepository, ProductRepository>();
builder.Services.AddTransient<IEmailService, EmailService>();
builder.Services.AddSingleton<ICacheService, MemoryCacheService>();

var app = builder.Build();
```

Here:
- `AddScoped<IProductRepository, ProductRepository>()` tells the container: whenever someone asks for `IProductRepository`, give them a `ProductRepository` instance that lives for the current HTTP request.
- `AddTransient` creates a new instance every time.
- `AddSingleton` creates a single instance shared by all requests.

#### Resolving Services

Once registered, services can be injected into constructors of controllers, middleware, other services, and even into views using `@inject`.

**Example: Injecting into a controller**

```csharp
public class ProductsController : Controller
{
    private readonly IProductRepository _productRepository;
    private readonly IEmailService _emailService;

    public ProductsController(IProductRepository productRepository, IEmailService emailService)
    {
        _productRepository = productRepository;
        _emailService = emailService;
    }

    // actions...
}
```

The container automatically provides the correct instances when the controller is created.

#### Injecting Services into Views

Sometimes you need a service directly in a view (e.g., a user service to check permissions). Use the `@inject` directive:

```html
@inject IUserService UserService

@if (UserService.IsAdmin(User))
{
    <a asp-action="Manage">Manage</a>
}
```

However, injecting services into views is less common; it's usually better to pass data through the model or view bag.

#### Multiple Implementations and Factory Pattern

If you have multiple implementations of the same interface, you can register them with a key (using a factory or by registering them as `IEnumerable<IMyService>`). The container will inject all registered instances.

```csharp
builder.Services.AddScoped<INotificationService, EmailNotificationService>();
builder.Services.AddScoped<INotificationService, SmsNotificationService>();
```

Then in a constructor:

```csharp
public HomeController(IEnumerable<INotificationService> notificationServices)
{
    // use all services
}
```

Or use a factory delegate for more control:

```csharp
builder.Services.AddScoped<INotificationService>(sp =>
{
    var config = sp.GetRequiredService<IConfiguration>();
    if (config["NotificationMode"] == "Email")
        return new EmailNotificationService();
    else
        return new SmsNotificationService();
});
```

---

### 7.3 Implementing the Repository Pattern

The **Repository pattern** is a design pattern that mediates between the domain and data mapping layers, acting like an in‑memory collection of domain objects. It provides a clean separation of concerns and makes your data access code more testable.

#### Why Abstract Data Access?

Without a repository, your controllers might directly use Entity Framework (or any other data access technology):

```csharp
public class ProductsController : Controller
{
    private readonly AppDbContext _context;

    public ProductsController(AppDbContext context)
    {
        _context = context;
    }

    public IActionResult Index()
    {
        var products = _context.Products.ToList();
        return View(products);
    }
}
```

This is already using DI (`AppDbContext` is injected), but it still couples the controller to Entity Framework and the database. If you later want to change to a different data source (e.g., a web API, a NoSQL database), you'd have to rewrite every controller action. Moreover, unit testing becomes harder because you'd need an in‑memory database or a complex mock.

The Repository pattern introduces an interface that represents collection-like operations:

```csharp
public interface IProductRepository
{
    Product GetById(int id);
    IEnumerable<Product> GetAll();
    void Add(Product product);
    void Update(Product product);
    void Delete(int id);
    bool Exists(int id);
}
```

Then you implement this interface using whatever data access technology you prefer. Controllers depend only on the interface.

#### Creating the Repository Interface

First, create an `IProductRepository` interface in a folder called `Repositories` or `Interfaces`:

```csharp
using MyFirstMvcApp.Models;

namespace MyFirstMvcApp.Repositories
{
    public interface IProductRepository
    {
        Task<IEnumerable<Product>> GetAllAsync();
        Task<Product> GetByIdAsync(int id);
        Task AddAsync(Product product);
        Task UpdateAsync(Product product);
        Task DeleteAsync(int id);
        Task<bool> ExistsAsync(int id);
    }
}
```

We've made the methods `async` to reflect real database operations.

#### Implementing the Repository (Simulated)

For now, we'll create a simple in‑memory implementation to keep the focus on DI. Later, in Chapter 8, we'll replace it with a real Entity Framework Core implementation.

```csharp
using MyFirstMvcApp.Models;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;

namespace MyFirstMvcApp.Repositories
{
    public class InMemoryProductRepository : IProductRepository
    {
        private readonly List<Product> _products = new List<Product>();
        private int _nextId = 1;

        public InMemoryProductRepository()
        {
            // Seed some data
            _products.Add(new Product { Id = _nextId++, Name = "Laptop", Price = 999.99m, Description = "High-performance laptop", CategoryId = 1, IsAvailable = true });
            _products.Add(new Product { Id = _nextId++, Name = "Mouse", Price = 19.99m, Description = "Wireless mouse", CategoryId = 1, IsAvailable = true });
            _products.Add(new Product { Id = _nextId++, Name = "Keyboard", Price = 49.99m, Description = "Mechanical keyboard", CategoryId = 1, IsAvailable = true });
        }

        public Task<IEnumerable<Product>> GetAllAsync() => Task.FromResult(_products.AsEnumerable());

        public Task<Product> GetByIdAsync(int id) => Task.FromResult(_products.FirstOrDefault(p => p.Id == id));

        public Task AddAsync(Product product)
        {
            product.Id = _nextId++;
            _products.Add(product);
            return Task.CompletedTask;
        }

        public Task UpdateAsync(Product product)
        {
            var existing = _products.FirstOrDefault(p => p.Id == product.Id);
            if (existing != null)
            {
                existing.Name = product.Name;
                existing.Price = product.Price;
                existing.Description = product.Description;
                existing.CategoryId = product.CategoryId;
                existing.IsAvailable = product.IsAvailable;
            }
            return Task.CompletedTask;
        }

        public Task DeleteAsync(int id)
        {
            var product = _products.FirstOrDefault(p => p.Id == id);
            if (product != null)
                _products.Remove(product);
            return Task.CompletedTask;
        }

        public Task<bool> ExistsAsync(int id) => Task.FromResult(_products.Any(p => p.Id == id));
    }
}
```

Now we have a repository that works with an in‑memory list. Later, we can swap it for a database-backed repository without changing any controller code.

#### Registering the Repository in DI

In `Program.cs`:

```csharp
builder.Services.AddScoped<IProductRepository, InMemoryProductRepository>();
```

Because we used `AddScoped`, a new repository instance will be created for each HTTP request. That's fine for our in‑memory list; but if we were using Entity Framework, scoped is exactly what we need (one `DbContext` per request).

---

### 7.4 Injecting Services into Controllers

Now we refactor our `ProductsController` to use the repository interface.

#### Constructor Injection

```csharp
using Microsoft.AspNetCore.Mvc;
using MyFirstMvcApp.Models;
using MyFirstMvcApp.Repositories;
using System.Threading.Tasks;

namespace MyFirstMvcApp.Controllers
{
    public class ProductsController : Controller
    {
        private readonly IProductRepository _productRepository;

        public ProductsController(IProductRepository productRepository)
        {
            _productRepository = productRepository;
        }

        // GET: /products
        public async Task<IActionResult> Index()
        {
            var products = await _productRepository.GetAllAsync();
            return View(products);
        }

        // GET: /products/details/5
        public async Task<IActionResult> Details(int id)
        {
            var product = await _productRepository.GetByIdAsync(id);
            if (product == null)
                return NotFound();
            return View(product);
        }

        // GET: /products/create
        public IActionResult Create()
        {
            // We'll need categories for dropdown, but that's separate
            return View();
        }

        // POST: /products/create
        [HttpPost]
        [ValidateAntiForgeryToken]
        public async Task<IActionResult> Create(Product product)
        {
            if (ModelState.IsValid)
            {
                await _productRepository.AddAsync(product);
                TempData["SuccessMessage"] = "Product created!";
                return RedirectToAction(nameof(Index));
            }
            return View(product);
        }

        // GET: /products/edit/5
        public async Task<IActionResult> Edit(int id)
        {
            var product = await _productRepository.GetByIdAsync(id);
            if (product == null)
                return NotFound();
            return View(product);
        }

        // POST: /products/edit/5
        [HttpPost]
        [ValidateAntiForgeryToken]
        public async Task<IActionResult> Edit(int id, Product product)
        {
            if (id != product.Id)
                return BadRequest();

            if (ModelState.IsValid)
            {
                await _productRepository.UpdateAsync(product);
                TempData["SuccessMessage"] = "Product updated!";
                return RedirectToAction(nameof(Index));
            }
            return View(product);
        }

        // GET: /products/delete/5
        public async Task<IActionResult> Delete(int id)
        {
            var product = await _productRepository.GetByIdAsync(id);
            if (product == null)
                return NotFound();
            return View(product);
        }

        // POST: /products/delete/5
        [HttpPost, ActionName("Delete")]
        [ValidateAntiForgeryToken]
        public async Task<IActionResult> DeleteConfirmed(int id)
        {
            await _productRepository.DeleteAsync(id);
            TempData["SuccessMessage"] = "Product deleted!";
            return RedirectToAction(nameof(Index));
        }
    }
}
```

Notice that the controller never mentions `InMemoryProductRepository`. It only knows `IProductRepository`. The DI container supplies the concrete instance at runtime.

#### The Power of Abstraction

If we later create a `SqlProductRepository` that uses Entity Framework, we only need to change the registration:

```csharp
builder.Services.AddScoped<IProductRepository, SqlProductRepository>();
```

All controllers continue to work unchanged. This is the essence of loose coupling.

#### Testing with Mocks

One of the biggest benefits of DI is testability. In a unit test, you can pass a mock implementation of `IProductRepository` to the controller.

Here's a simple example using xUnit and Moq (a popular mocking library):

```csharp
using Moq;
using MyFirstMvcApp.Controllers;
using MyFirstMvcApp.Models;
using MyFirstMvcApp.Repositories;
using System.Collections.Generic;
using System.Threading.Tasks;
using Xunit;

public class ProductsControllerTests
{
    [Fact]
    public async Task Index_ReturnsViewWithProducts()
    {
        // Arrange
        var mockRepo = new Mock<IProductRepository>();
        mockRepo.Setup(repo => repo.GetAllAsync())
                .ReturnsAsync(new List<Product> { new Product { Id = 1, Name = "Test" } });

        var controller = new ProductsController(mockRepo.Object);

        // Act
        var result = await controller.Index();

        // Assert
        var viewResult = Assert.IsType<ViewResult>(result);
        var model = Assert.IsAssignableFrom<IEnumerable<Product>>(viewResult.Model);
        Assert.Single(model);
    }
}
```

We don't need a database; we simply configure the mock to return a predetermined list. This test is fast, reliable, and isolated.

---

### 7.5 Advanced DI Scenarios (Brief)

#### Injecting Multiple Services with `IEnumerable<T>`

As mentioned earlier, you can inject all registered implementations of an interface:

```csharp
public class NotificationService : INotificationService
{
    private readonly IEnumerable<INotificationChannel> _channels;

    public NotificationService(IEnumerable<INotificationChannel> channels)
    {
        _channels = channels;
    }

    public async Task SendAsync(string message)
    {
        foreach (var channel in _channels)
            await channel.SendAsync(message);
    }
}
```

#### Using Factory Delegates

Sometimes you need to create a service conditionally or with runtime parameters. You can register a factory delegate:

```csharp
builder.Services.AddScoped<IPaymentService>(sp =>
{
    var config = sp.GetRequiredService<IConfiguration>();
    var provider = config["PaymentProvider"];
    return provider switch
    {
        "Stripe" => new StripePaymentService(),
        "PayPal" => new PayPalPaymentService(),
        _ => throw new InvalidOperationException()
    };
});
```

#### Service Locator Anti-Pattern

Avoid using `IServiceProvider` directly in your classes (by injecting `IServiceProvider` and calling `GetService`). This hides dependencies and makes testing difficult. Stick to constructor injection.

---

### Summary

In this chapter, you learned the critical concepts of Dependency Injection and the Repository pattern:

- **Dependency Injection** decouples your classes from their dependencies, promoting testability and flexibility.
- ASP.NET Core's built‑in DI container supports three lifetimes: **Transient**, **Scoped**, and **Singleton**.
- You register services in `Program.cs` using methods like `AddScoped<T>`.
- The **Repository pattern** abstracts data access behind an interface, allowing you to swap implementations and simplify testing.
- Constructor injection is the standard way to receive dependencies in controllers and other services.
- With DI and repositories, your code becomes cleaner, more maintainable, and easily testable.

**Exercise:**

1. Create an `ICategoryRepository` interface with methods like `GetAllAsync()`, `GetByIdAsync()`, etc.
2. Implement an `InMemoryCategoryRepository` similar to the product repository.
3. Register it in DI as scoped.
4. Modify the `ProductsController.Create` GET and POST actions to use the category repository to populate the category dropdown. (You'll need to inject `ICategoryRepository` into the controller.)
5. Write a simple unit test (using a mocking framework or manual fake) for the `Create` POST action that verifies `AddAsync` is called when the model is valid.

In the next chapter, **"Entity Framework Core,"** we'll replace our in‑memory repositories with a real database. You'll learn how to set up `DbContext`, create migrations, and perform CRUD operations using Entity Framework Core—all while maintaining the clean separation we've established.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='6. model_validation.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='8. entity_framework_core.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
